In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, classification_report

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From here, we have pred, labels, and confidences from the CNN

# START HERE WITH PREPROCESSED FILES

In [ ]:
import pandas as pd
import ast

reconstruct = pd.read_csv("rec.csv")

# Convert the feature column from string to list of integers
reconstruct["feature"] = reconstruct["feature"].apply(ast.literal_eval)

print(reconstruct.head())  # Check the first few rows
print(type(df["feature"][0])) 

                                      feature  label
0  [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]      0
1  [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0]      1
2  [1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]      1
3  [1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]      1
4  [1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1]      1
<class 'list'>


#### Manual Error Rate Calculation

In [17]:
print(reconstruct['feature'][0])
print(reconstruct['label'][0]) #this is B (U == U*)

manual_error = []
arr_len = len(reconstruct['feature'][0])
tot_enumerations = 2**len(reconstruct['feature'][0])

[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
0


In [32]:
import numpy as np
from itertools import product
from tqdm import tqdm


def manual_error_rate(df, model, proba=False, diag=False):
    features = df['feature']
    labels = df['label']

    arr_len = len(features[0])
    tot_enumerations = 2**arr_len

    permutations = list(product([0, 1], repeat=arr_len))

    results = []
    results_p = []

    for perm in tqdm(permutations, total=tot_enumerations, desc="Processing permutations"):
        perm_array = np.array(perm)
        count = 0
        error_count = 0

        for feature, label in zip(features, labels):
            if np.array_equal(feature, perm_array):
                count += 1
                if label == 0:
                    error_count += 1

        if count > 0:
          error_rate = (error_count / count)
          results.append([perm, error_rate])
          proba_score = 0
          if proba:
            proba_score = model.predict_proba(np.array(perm).reshape(1, -1))[0][0]
            results_p.append([perm, proba_score])
          if diag: print(f"permutation {perm}: manual error rate is {error_rate*100:.2f}%, proba_score is {proba_score*100:.2f}% appears {count} times")

    results_array = np.array(results, dtype=object)
    results_array_p = np.array(results_p, dtype=object)

    if proba: return results_array, results_array_p
    else: return results_array


### Manual Error Rate of 100k Set

In [20]:
manual_tot = manual_error_rate(reconstruct, rf)

Processing permutations: 100%|██████████| 16384/16384 [22:11<00:00, 12.31it/s]


In [ ]:
#save manual_tot, it is a numpy array
np.save('manual_tot.npy', manual_tot)

### Manual and Proba Error Rate of 10k Set

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(list(reconstruct['feature']), reconstruct['label'])

rf.predict_proba(np.array(reconstruct['feature'][0]).reshape(1, -1))

In [ ]:
sample_rec = reconstruct.sample(n=10000, random_state=42)
#reset index
sample_rec = sample_rec.reset_index(drop=True)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(list(sample_rec['feature']), sample_rec['label'])

manual_sample, proba_sample = manual_error_rate(sample_rec, rf, proba=True, diag=False)

np.save('manual_sample.npy', manual_sample)
np.save('proba_sample.npy', proba_sample)

Processing permutations: 100%|██████████| 16384/16384 [04:53<00:00, 55.74it/s]


#### R^2 Values

In [3]:
proba_sample = np.load('proba_sample.npy', allow_pickle=True)
manual_sample = np.load('manual_sample.npy', allow_pickle=True)
manual_tot = np.load('manual_tot.npy', allow_pickle=True)

In [4]:
proba_sample

array([[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 0.003081969274486076],
       [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1), 0.9881310502789765],
       [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0), 0.0],
       ...,
       [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1), 0.0941904761904762],
       [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0), 0.05371428571428571],
       [(1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0), 0.5415576645576643]],
      dtype=object)

In [5]:
# only keep permutations that are present in sample_rec
import numpy as np

# Convert lists of features in proba_sample and manual_sample into sets for quick lookup
proba_features = {tuple(entry[0]) for entry in proba_sample}
manual_sample_features = {tuple(entry[0]) for entry in manual_sample}

# Find the intersection of features in both proba_sample and manual_sample
common_features = proba_features & manual_sample_features

# Filter manual_tot to keep only entries with features in common_features
manual_tot_sheared = np.array([entry for entry in manual_tot if tuple(entry[0]) in common_features], dtype=object)

print(manual_tot_sheared)


[[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 0.003424657534246575]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1) 0.9859154929577465]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0) 0.0]
 ...
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1) 0.0]
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0) 0.0]
 [(1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0) 0.5]]


In [7]:
print(manual_tot_sheared)

[[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 0.003424657534246575]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1) 0.9859154929577465]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0) 0.0]
 ...
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1) 0.0]
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0) 0.0]
 [(1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0) 0.5]]


In [8]:
print(manual_sample)

[[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 0.017241379310344827]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1) 1.0]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0) 0.0]
 ...
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1) 0.0]
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0) 0.0]
 [(1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0) 1.0]]


In [9]:
print(proba_sample)

[[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 0.003081969274486076]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1) 0.9881310502789765]
 [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0) 0.0]
 ...
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1) 0.0941904761904762]
 [(1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0) 0.05371428571428571]
 [(1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0) 0.5415576645576643]]


In [10]:
manual_tot_sheared[:, 1]

array([0.003424657534246575, 0.9859154929577465, 0.0, ..., 0.0, 0.0, 0.5],
      dtype=object)

In [38]:
from sklearn.metrics import r2_score

#r2 for proba_sample and manual_tot
print(r2_score(manual_tot_sheared[:, 1], proba_sample[:, 1]))

#r2 for manual_sample and manual_tot
print(r2_score(manual_tot_sheared[:, 1], manual_sample[:, 1]))

0.9585035263308146
0.2822243456534057
